检查是否处于虚拟环境

In [1]:
!powershell -Command "Get-Command python" | findstr venv

Application     python.exe                                         3.11.91... e:\My Workspace\neoPythonModule\.venv\...


导入库

- `os` - 与操作系统交互
- `re` - 正则表达式操作
- `json` - 处理JSON格式
- `requests` - 网络操作
- `biliapis` - 调用B站API
- `bilicodes` - B站API使用的枚举值

> 按照[PEP 8](https://peps.python.org/pep-0008/#imports)，导入库的顺序应该是 `标准库 - 三方库 - 本地库`，且每组间有一个空行

In [2]:
import os
import re
import json

import requests

import biliapis
import bilicodes

就先暂时使用字符串代替一下用户输入吧。因为这里只是梳理一下流程。

使用正则表达式提取用户输入中的`avID`或`bvID`。（代码中所示的表达式可能不是最优解，请自行判断）

> ~~其实现在复杂的正则表达式更多让AI帮忙写~~

> bvID优先，因为bvID中有时候会碰巧出现`avXXX`这样的字符组合。

将用于匹配`avID`、`bvID`的正则表达式字符串预编译。

In [3]:
REGEX_BVID = re.compile(r"(BV[a-zA-Z0-9]{10})", re.I)
REGEX_AVID = re.compile(r"av([0-9]+)", re.I)

In [4]:
user_input = "https://www.bilibili.com/video/BV1q3411E7P7/"
bvids = REGEX_BVID.findall(user_input)
avids = REGEX_AVID.findall(user_input)
ids = {}
if bvids:
    ids["bvid"] = bvids[0]
elif avids:
    ids["avid"] = int(avids[0])
else:
    # 这里直接raise，写在函数里时记得用return代替
    raise RuntimeError("No valid id detected")

调用API请求数据

看不懂这里的`**ids`？去看[补注](../notes.md#任意数量参数)！

In [5]:
video_data = biliapis.get_video_detail(**ids)
print(json.dumps(video_data, ensure_ascii=False, indent=4))

{
    "code": 0,
    "message": "0",
    "ttl": 1,
    "data": {
        "bvid": "BV1q3411E7P7",
        "aid": 423736658,
        "videos": 2,
        "tid": 201,
        "tname": "科学科普",
        "copyright": 1,
        "pic": "http://i1.hdslb.com/bfs/archive/e4cae24d721b65e59176b701f9e338bc9be536d8.jpg",
        "title": "一位学生在10分钟内读完了31页线性代数，这是他大脑发生的变化",
        "pubdate": 1643557557,
        "ctime": 1643557557,
        "desc": "A Student Finished 31 Pages Of Linear Algebra In 10 Minutes.\r\nThis Is What Happens To His Brain.\r\n模仿chubbyemu的整活视频（）",
        "desc_v2": [
            {
                "raw_text": "A Student Finished 31 Pages Of Linear Algebra In 10 Minutes.\r\nThis Is What Happens To His Brain.\r\n模仿chubbyemu的整活视频（）",
                "type": 1,
                "biz_id": 0
            }
        ],
        "state": 0,
        "duration": 398,
        "mission_id": 351469,
        "rights": {
            "bp": 0,
            "elec": 0,
            "download": 1,
       

从中我们可以得到`cid`（我猜是`content ID`的简写），接下来我们可以用`cid`取到视频流。当然也可以做一些别的事情比如下载弹幕文件（xml格式）。

有相当一部分视频拥有多个分P（见上框运行结果中的`$.data.pages`），每个分P都对应着一个`cid`，可以让用户再做一个选择。

> 对于JSON格式，也有像是`XPath`对于XML那样的`JSONPath`表达式可以用，可以用于处理复杂的json文件。

In [6]:
pages: list[dict] = video_data["data"]["pages"]

user_choice = 0 # 假定用户选择了第1个分P
cid = pages[user_choice]["cid"]
stream = biliapis.get_video_stream_dash(cid=cid, **ids)
print(json.dumps(stream, ensure_ascii=False, indent=4))

{
    "code": 0,
    "message": "0",
    "ttl": 1,
    "data": {
        "from": "local",
        "result": "suee",
        "message": "",
        "quality": 32,
        "format": "flv480",
        "timelength": 198738,
        "accept_format": "hdflv2,hdflv2,flv,flv720,flv480,mp4",
        "accept_description": [
            "超清 4K",
            "高清 1080P+",
            "高清 1080P",
            "高清 720P",
            "清晰 480P",
            "流畅 360P"
        ],
        "accept_quality": [
            120,
            112,
            80,
            64,
            32,
            16
        ],
        "video_codecid": 7,
        "seek_param": "start",
        "seek_type": "offset",
        "dash": {
            "duration": 199,
            "minBufferTime": 1.5,
            "min_buffer_time": 1.5,
            "video": [
                {
                    "id": 32,
                    "baseUrl": "https://cn-cq-gd-live-03.bilivideo.com/upgcxcode/70/51/498005170/498005170_nb2-1-30032.m4

~~又取到流了家人们~~

在字典`$.data.dash`中，存在两个列表`video`和`audio`，它们之中的字典包含了各个质量的媒体流。这就是为什么客户端能根据网络质量在各个画质间自动**快速**切换。

让我们随便取一个质量的视频流，看看是怎么个事。

In [7]:
video_streams = stream["data"]["dash"]["video"]
audio_streams = stream["data"]["dash"]["audio"]

print(json.dumps(video_streams[0], ensure_ascii=False, indent=4))

{
    "id": 32,
    "baseUrl": "https://cn-cq-gd-live-03.bilivideo.com/upgcxcode/70/51/498005170/498005170_nb2-1-30032.m4s?e=ig8euxZM2rNcNbdlhoNvNC8BqJIzNbfqXBvEqxTEto8BTrNvN0GvT90W5JZMkX_YN0MvXg8gNEV4NC8xNEV4N03eN0B5tZlqNxTEto8BTrNvNeZVuJ10Kj_g2UB02J0mN0B5tZlqNCNEto8BTrNvNC7MTX502C8f2jmMQJ6mqF2fka1mqx6gqj0eN0B599M=&uipk=5&nbs=1&deadline=1721647531&gen=playurlv2&os=bcache&oi=2101216138&trid=0000f27869831cf14549b8700398999a40d6u&mid=0&platform=pc&og=cos&upsig=76ad04e8122b6db10a2d1ff512741576&uparams=e,uipk,nbs,deadline,gen,os,oi,trid,mid,platform,og&cdnid=20320&bvc=vod&nettype=0&orderid=0,3&buvid=&build=0&f=u_0_0&agrr=1&bw=54104&logo=80000000",
    "base_url": "https://cn-cq-gd-live-03.bilivideo.com/upgcxcode/70/51/498005170/498005170_nb2-1-30032.m4s?e=ig8euxZM2rNcNbdlhoNvNC8BqJIzNbfqXBvEqxTEto8BTrNvN0GvT90W5JZMkX_YN0MvXg8gNEV4NC8xNEV4N03eN0B5tZlqNxTEto8BTrNvNeZVuJ10Kj_g2UB02J0mN0B5tZlqNCNEto8BTrNvNC7MTX502C8f2jmMQJ6mqF2fka1mqx6gqj0eN0B599M=&uipk=5&nbs=1&deadline=1721647531&gen=playurlv

可以看到里面除了媒体流地址，还有各种各样的提供给解码器（或者说播放器？）的信息。不过既然我们只需要下载它们，那就只需要关心地址就好（吗？）

对照`bilicodes`中的信息，我们可以知道这个流是什么质量的。（其实从编码参数中的分辨率也能看出来）

In [8]:
print(bilicodes.stream_dash_video_quality[video_streams[0]["id"]])

480P


得到了媒体流地址后，我们就可以开始下载了。需要用到`requests`的流式传输。

> 下载流时记得带上指向B站域名的`Referer`请求头，服务器会检验这个字段。

这里我们假设用户选择了当前目录（`./`）作为保存位置，并对于音频和视频都选择了质量最好的流。

从标题和分P名生成文件名，使用正则表达式去除不能作为文件名的字符

In [9]:
# 假定用户选择了最高质量的流
video_stream = max(video_streams, key=lambda x: x["height"])
audio_stream = max(audio_streams, key=lambda x: x["bandwidth"])
url_v = video_stream["base_url"]
url_a = audio_stream["base_url"]

save_dir = "./"
title = video_data["data"]["title"] + "_" + pages[user_choice]["part"]
# 通过cid生成临时文件名
tmpfile_au = os.path.join(save_dir, "%d_au.m4a" % cid)
tmpfile_vi = os.path.join(save_dir, "%d_vi.m4v" % cid)
# 通过title和分P名生成最终文件名
file_final = os.path.join(save_dir, re.sub(r"[<>:\'\"/\\|?*]", "_", title) + ".mp4")
# 视频流
with open(tmpfile_vi, "wb+") as fp:
    with requests.get(url_v, stream=True, headers=biliapis.DEFAULT_HEADERS) as resp:
        for chunk in resp.iter_content(chunk_size=8192):
            if chunk:
                fp.write(chunk)
# 音频流
with open(tmpfile_au, "wb+") as fp:
    with requests.get(url_a, stream=True, headers=biliapis.DEFAULT_HEADERS) as resp:
        for chunk in resp.iter_content(chunk_size=8192):
            if chunk:
                fp.write(chunk)

通过命令行调用ffmpeg进行合流，合流完毕后删除音频流与视频流

关于ffmpeg从命令行的调用，参见[ffmpeg官方文档](https://www.ffmpeg.org/documentation.html)等网站。也可使用另外的库比如[`python-ffmpeg`](https://kkroening.github.io/ffmpeg-python/)。

In [10]:
os.system(
    'ffmpeg -i "%s" -i "%s" -vcodec copy -acodec copy "%s"'
    % (tmpfile_vi, tmpfile_au, file_final)
)
os.remove(tmpfile_vi)
os.remove(tmpfile_au)

尽管还有**亿些**需要改善的细节，但这就是**最基本**的视频下载流程了。

让我们回到[README](../README.md#成型)中，细节放在了[补注](../notes.md)中。